# Site Splitting
### Author: Max Sop
The script in this notebook attempts to address one of the limitations of the route optimization program by splitting sites whose total weights (KGs) exceed a truck load capacity. Further development and adjustments would be required based on the Tech team suggestions. This should then be written as a Python script that sits on top of the route optimization program. So this is just a temporary solution. Ideally the LP optimizer needs to be rewritten in order to address all lingering issues such as cases where the solver doesn't return the expected results or to easily embed this program within the solver.

For contextual or further understanding of the problem, please get in touch with Tobias Krause [tobias.krause@oneacrefund.org ]

In [1]:
#Load basic libraries
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
distance_df     = pd.read_excel('DistanceMatrix (2).xlsx')
client_input_df = pd.read_excel('Client Inputs.xlsx', sheet_name='Client Inputs')

In [3]:
client_input_grp = client_input_df.groupby(['SiteName','GroupName'])['TotalKgWeight'].agg(sum).reset_index(level=['SiteName',"GroupName"])

In [4]:
client_input_grp['SiteWeight'] = client_input_grp.groupby(['SiteName'])['TotalKgWeight'].transform(sum)
client_input_grp.sort_values(by=['SiteName' ,'TotalKgWeight'], ascending=True, inplace=True)
client_input_grp['GroupWeightRunningTotal'] = client_input_grp.groupby(['SiteName'])['TotalKgWeight'].transform(pd.Series.cumsum)

In [5]:
client_input_grp[client_input_grp['SiteName'] == 'Balek'].reset_index(drop=True)

,SiteName,GroupName,TotalKgWeight,SiteWeight,GroupWeightRunningTotal
0,Balek,Bidii,107.800,11797.28,107.800
1,Balek,Cheptagum,108.700,11797.28,216.500
2,Balek,Central,166.550,11797.28,383.050
3,Balek,Pluto,182.550,11797.28,565.600
4,Balek,kaple,197.250,11797.28,762.850
5,Balek,kapchelogoi,217.600,11797.28,980.450
6,Balek,Koshin,235.750,11797.28,1216.200
7,Balek,Kabirget,237.500,11797.28,1453.700
8,Balek,Tembwet,250.850,11797.28,1704.550
9,Balek,Soingo,257.250,11797.28,1961.800


In [259]:
def loop_routine(weight, max_truck_size, df, lst, ind, site_name, i):
        truck_load = df['TotalKgWeight'][:ind].sum()
        lst.append({'Sites': site_name + '_' + str(i), 'GroupList': list(df['GroupName'][:ind]), 'TruckLoad': truck_load})
        df = df.iloc[ind:].reset_index(drop=True)
        df['GroupWeightRunningTotal'] = df['TotalKgWeight'].cumsum()
        return df 

In [281]:
list(client_input_grp.SiteName.unique())

'Balek'

In [8]:
def split_site_group(df=client_input_grp, max_truck_size=10000):
    lst = []
    site_names =list(client_input_grp.SiteName.unique())
    for site_name in site_names:
        df = client_input_grp.loc[client_input_grp['SiteName'] == site_name, ['SiteWeight', 'GroupName', 'GroupWeightRunningTotal', 'TotalKgWeight']].reset_index(drop=True)
        site_weight = df['SiteWeight'][0]
        if(site_weight > max_truck_size): 
            i = 1
            for ind, weight in enumerate(df['GroupWeightRunningTotal']):
                if(weight >= max_truck_size):
                    truck_load = df['TotalKgWeight'][:ind].sum()
                    if (truck_load == 0): break
                    lst.append({'Sites': site_name + '_' + str(i), 'GroupList': list(df['GroupName'][:ind]), 'TruckLoad': truck_load})
                    i+=1
                    df = df.iloc[ind:]
                    if(df.shape[0] == 1):
                        lst.append({'Sites': site_name + '_' + str(i), 'GroupList': list(df['GroupName']), 'TruckLoad': df['TotalKgWeight'].values[0]})
        else:
            lst.append({'Sites': site_name, 'GroupList': list(df['GroupName']), 'TruckLoad': site_weight}) 
    return pd.DataFrame(lst)

In [9]:
split_site_group()

,GroupList,Sites,TruckLoad
0,"[Bidii, Cheptagum, Central, Pluto, kaple, kapc...",Balek_1,9837.618
1,"[Saptet, midpoint]",Balek_2,1959.662
2,"[ebenezer, Amani, Emityot, kerundut, SIRIKWA, ...",Chebilat_1,8764.580
3,[Canan],Chebilat_2,1431.318
4,"[Pondet, Ngetumio, Munantet, Tokjin, Sibayan, ...",Chebirbelek,6684.370
5,"[Greatvision, star, Kiletien, Kopor, ronkos, b...",Chebole_1,9182.454
6,"[revelation, Asis, Kapkoin, Kiburburen]",Chebole_2,3738.845
7,"[Sachangwany, Kulwet, CHEPTAGUM, Kapmugeni, Ch...",Chebongi,8916.568
8,"[Corner, kona, kolongei, Cheptangulgel, Chomny...",Chesilyot,5912.712
9,"[KONDAMET, Mizizi, KOPTIGEI, Sigingit, Extreme...",Kapchepkoro,4517.606


In [10]:
split_sites =  split_site_group()

In [11]:
def transform_distance(distance_df=distance_df, split_sites=split_sites):
    
    distance_df.columns = [column.replace(' (SOT)', '') for column in distance_df.columns]
    distance_df['Drops'] = distance_df['Drops'].apply(lambda x: x.replace(' (SOT)', ''))
    distance_df.set_index('Drops', inplace=True)
    site_columns = [site.split('_')[0] for site in split_sites.Sites if '_' in site]
    distance_columns = [column for column in distance_df.columns]
    
    df_column = pd.DataFrame()
    df_row = pd.DataFrame()
    
    for column in distance_columns:
        if column not in site_columns:
            df_column[column] = distance_df[column]
        else:
            for i in range(1, 3):
                col_name = column + '_' + str(i)
                df_column[col_name] = distance_df[column] 

    df_column = df_column.transpose()
    
    for column in distance_columns:
        if column not in site_columns:
            df_row[column] = df_column[column]
        else:
            for i in range(1, 3):
                col_name = column + '_' + str(i)
                df_row[col_name] = df_column[column] 
                
    return df_row

In [12]:
transform_distance()

,Balek_1,Balek_2,Chebilat_1,Chebilat_2,Chebirbelek,Chebole_1,Chebole_2,Chebongi,Chesilyot,Kapchepkoro,...,Kipajit,Kiriba,Koimeret,Mabwaita,Makimeny,Rongena,Sigorian,Siroin,Yaganek,Nyansiongo
Balek_1,0.000,0.000,18.489,18.489,6.706,17.232,17.232,16.455,2.153,13.191,...,18.808,40.944,31.767,28.008,25.198,24.621,32.434,6.382,9.909,29.707
Balek_2,0.000,0.000,18.489,18.489,6.706,17.232,17.232,16.455,2.153,13.191,...,18.808,40.944,31.767,28.008,25.198,24.621,32.434,6.382,9.909,29.707
Chebilat_1,18.489,18.489,0.000,0.000,25.195,20.814,20.814,6.998,16.335,9.280,...,8.738,44.527,35.350,12.488,28.780,9.101,25.468,24.871,9.959,11.218
Chebilat_2,18.489,18.489,0.000,0.000,25.195,20.814,20.814,6.998,16.335,9.280,...,8.738,44.527,35.350,12.488,28.780,9.101,25.468,24.871,9.959,11.218
Chebirbelek,6.706,6.706,25.195,25.195,0.000,14.030,14.030,23.161,8.860,19.251,...,25.514,37.028,27.851,34.714,21.282,31.327,28.518,0.324,15.969,36.413
Chebole_1,17.232,17.232,20.814,20.814,14.030,0.000,0.000,18.780,19.385,14.429,...,12.567,24.180,14.951,30.333,8.382,26.946,15.618,13.705,11.147,32.032
Chebole_2,17.232,17.232,20.814,20.814,14.030,0.000,0.000,18.780,19.385,14.429,...,12.567,24.180,14.951,30.333,8.382,26.946,15.618,13.705,11.147,32.032
Chebongi,16.455,16.455,6.998,6.998,23.161,18.780,18.780,0.000,14.302,7.246,...,7.317,42.493,33.316,16.517,26.746,13.129,33.983,22.837,7.925,18.216
Chesilyot,2.153,2.153,16.335,16.335,8.860,19.385,19.385,14.302,0.000,11.105,...,16.655,43.097,33.920,25.854,27.351,22.467,34.587,8.535,11.784,27.553
Kapchepkoro,13.259,13.259,7.265,7.265,19.251,14.429,14.429,5.231,11.105,0.000,...,5.374,38.141,28.964,16.784,22.395,13.396,29.631,18.927,3.574,18.483


In [ ]:
# def split_site_group(df=client_input_grp, max_truck_size=5000):
#     lst = []
#     site_names = list(client_input_grp.SiteName.unique())
#     for site_name in site_names:
#         df = client_input_grp.loc[client_input_grp['SiteName'] == site_name, ['SiteWeight', 'GroupName', 'GroupWeightRunningTotal', 'TotalKgWeight']].reset_index(drop=True)
#         site_weight = df['SiteWeight'][0]
#         if(site_weight > max_truck_size): 
#             i = 1
#             for ind, weight in enumerate(df['GroupWeightRunningTotal']):
#                 if(weight >= max_truck_size):
#                     truck_load = df['TotalKgWeight'][:ind].sum()
#                     if (truck_load == 0): break
#                     lst.append({'Sites': site_name + '_' + str(i), 'GroupList': list(df['GroupName'][:ind]), 'TruckLoad': truck_load})
#                     i+=1
#                     df = df.iloc[ind:]
#                     if(df.shape[0] == 1):
#                         lst.append({'Sites': site_name + '_' + str(i), 'GroupList': list(df['GroupName']), 'TruckLoad': df['TotalKgWeight'].values[0]})
#         else:
#             lst.append({'Sites': site_name, 'GroupList': list(df['GroupName']), 'TruckLoad': site_weight}) 
#     return pd.DataFrame(lst)